# Datos

## Idea

Basados en tres bases de datos:

1. Experiencia previa
2. Participación económica
3. Puestos en otras compañías

Para el periodo de 2002--2018 (17 años)

## Unir tablas

Usaremos, para cada año, los usuarios que tienen valores de participación económica,
puestos en otras compañías, y experiencia previa. Posteriormente se aplicaran los filtros que se piensen necesarios. Sin embargo, como se va a comparar con información de compra/venta de contratos públicos, únicamente se incluirá la experiencia de los funcionarios en compañías **privadas**.

## Primer año (2002)

Antes de obtener la tabla final, empezaremos a explorar las tablas de 2002 y dejar la metodología lista para los demás años.

La tabla `2002_generalesEncargo_DBP.csv` cuenta con la información principal de cada servidor público. Se usará para hacer el *merge* con las demás tablas. Todas las tablas cuentan con la variable **ACUSE** qué servirá de *id* para juntarlas.

La información de las **participaciones económicas** no fue incorporado sino hasta 2015, y la información de **puestos en otras compañías** hasta 2016. Por lo que durante el periodo 2002--2014 sólamente podremos usar la variable de **experiencia previa**.

## Variables globales

Antes de empezar, cargamos las librerías y definimos los caminos a los datos que se usarán de aquí en adelante.

In [1]:
import os
import re
import hashlib
from os.path import join
import pandas as pd
import numpy as np

In [2]:
DATA = "/home/rdora/declaranet/data"
# Tablas
GENERAL = "_generalesEncargo_DBP.csv"
EXP = "_experiencia_DBP.csv"
ECONO = "_participaciones_econo_DBP.csv"
PUESTO = "_puesto_cargo_com_DBP.csv"

In [3]:
path_general = join(DATA, "2002", "2002" + GENERAL)
path_exp = join(DATA, "2002", "2002" + EXP)
df_exp = pd.read_csv(path_exp)
df_general = pd.read_csv(path_general)

### Selección de variables
Las tablas cuentan con diversa información distribuida en variables. Para nuestro objetivo, unas son más importantes que otras.

#### General
Variables importantes:
1. acuse (ID)
2. nombre
3. tipo_declaracion
4. fecha_envio
5. puesto
6. dependencia
7. area_adscripcion
8. honorarios
9. maximo_grado_estudios

#### Experiencia
Variables importantes:
1. acuse
2. sector
3. institucion_empresa
4. area_o_unidad_administrativa (**NOTA**: typo en "adminsitrativa")
5. puesto
6. ingreso_egreso

In [4]:
COLS_GENERAL = ["ACUSE",
                "NOMBRE",
                "TIPO_DECLARACION",
                "FECHA_ENVIO",
                "PUESTO",
                "DEPENDENCIA",
                "AREA_ADSCRIPCION",
                "HONORARIOS",
                "MAXIMO_GRADO_ESTUDIOS"]
COLS_EXP = ["ACUSE",
            "SECTOR",
            "INSTITUCION_EMPRESA",
            "AREA_O_UNIDAD_ADMINSITRATIVA",
            "PUESTO",
            "INGRESO_EGRESO"]

In [5]:
df_general = df_general[COLS_GENERAL]
df_exp = df_exp[COLS_EXP]

In [6]:
df = pd.merge(df_exp,
             df_general,
             how="left",
             on="ACUSE")

## Segundo año (2003)

Exploración del segundo año, sobretodo para ver como se manejan los funcionarios públicos repetidos con 2002.

In [7]:
path_general = join(DATA, "2003", "2003" + GENERAL)
path_exp = join(DATA, "2003", "2003" + EXP)
df_exp_2003 = pd.read_csv(path_exp, usecols=COLS_EXP)
df_general_2003 = pd.read_csv(path_general, usecols=COLS_GENERAL)

En cuanto a experiencia, vamos a ver cuantos puestos están repetidos con 2002.

In [8]:
acuses_2002 = set(df_exp['ACUSE'].unique())
acuses_2003 = set(df_exp_2003["ACUSE"].unique())
print("Acuses repetidos: ", len(acuses_2002 & acuses_2003))

Acuses repetidos:  0


Para ver cuantas experiencias repetidas tenemos, primero tenemos que pasar las columnas a `string` y después crear un identificador único.

In [105]:
df_exp = df_exp.astype(str)
df_exp_2003 = df_exp_2003.astype(str)

In [55]:
df_exp["id"] = df_exp.iloc[:,1:].agg("-".join, axis=1)

df_exp_2003["id"] = df_exp_2003.iloc[:, 1:].agg("-".join, axis=1)

In [59]:
ids_2002 = set(df_exp['id'].unique())
ids_2003 = set(df_exp_2003["id"].unique())
print("pocentaje de ids repetidos: ", len(ids_2002 & ids_2003) / len(ids_2002))

pocentaje de ids repetidos:  0.6425877175749479


Vamos a crear un identificador único que sea un hash (id del mismo tamaño) y provar que obtenemos el mismo porcentaje de repetidos.

In [82]:
df_exp['hash'] = df_exp.iloc[:,1:].apply(
    lambda x: hashlib.md5("".join(x).encode()).hexdigest(), axis=1)
df_exp_2003['hash'] = df_exp_2003.iloc[:,1:].apply(
    lambda x: hashlib.md5("".join(x).encode()).hexdigest(), axis=1)

In [83]:
hashs_2002 = set(df_exp['hash'].unique())
hashs_2003 = set(df_exp_2003["hash"].unique())
print("pocentaje de hashs repethashos: ", len(hashs_2002 & hashs_2003) / len(hashs_2002))

pocentaje de hashs repethashos:  0.6425877175749479


#### General

Vamos a ver los repetidos como se comportan en la tabla general

In [87]:
acuses_2002 = set(df_general['ACUSE'].unique())
acuses_2003 = set(df_general_2003["ACUSE"].unique())
print("Acuses repetidos: ", len(acuses_2002 & acuses_2003))

Acuses repetidos:  0


In [106]:
df_general = df_general.astype(str)
df_general_2003 = df_general_2003.astype(str)

In [97]:
df_general_2003[df_general_2003.NOMBRE=="LEONARDO DE LA CRUZ AGUILAR"]

,ACUSE,NOMBRE,TIPO_DECLARACION,FECHA_ENVIO,PUESTO,DEPENDENCIA,AREA_ADSCRIPCION,HONORARIOS,MAXIMO_GRADO_ESTUDIOS,hash
94124,200302031812341386506,LEONARDO DE LA CRUZ AGUILAR,CONCLUSION,03/02/2003,ASESOR,CONSEJERIA JURIDICA DEL EJECUTIVO FEDERAL,CONSEJERIA ADJUNTA,NO,LICENCIATURA,7e04d27ed4b92bab66bfde035c65a32e


In [94]:
df_general.head()

,ACUSE,NOMBRE,TIPO_DECLARACION,FECHA_ENVIO,PUESTO,DEPENDENCIA,AREA_ADSCRIPCION,HONORARIOS,MAXIMO_GRADO_ESTUDIOS,hash
0,6482,LEONARDO DE LA CRUZ AGUILAR,INICIAL,08/05/2002,JEFE DE DEPARTAMENTO,CONSEJERIA JURIDICA DEL EJECUTIVO FEDERAL,CONSEJERIA JURIDICA,NO,LICENCIATURA,2422816d827f0fcdb68d8d4f044f92bf
1,200205230944131094591,VICTOR MANUEL CORONA NORIEGA,MODIFICACION,23/05/2002,ENLACE ADMINISTRATIVO,SECRETARIA DE MEDIO AMBIENTE Y RECURSOS NATURALES,INSTITUTO NACIONAL DE ECOLOGIA DIRECCION GENER...,NO,LICENCIATURA,feaef61127bfb19422f0489154a52a5f
2,200205291619561249043,JESUS ARMANDO GIL CASAMAYOR,MODIFICACION,29/05/2002,SUPERVISOR DE AUDITORIA FISCAL,SERVICIO DE ADMINISTRACION TRIBUTARIA,ADMINISTRACION LOCAL DE AUDITORIA FISCAL DE CH...,NO,LICENCIATURA,6fcf80d4ae82c3893def04958f1607cf
3,9497,MANUEL CORTÃ¿S AGUILAR,MODIFICACION,09/05/2002,JEFE DE DEPARTAMENTO,PROCURADURIA FEDERAL DEL CONSUMIDOR (PROFECO).,UNIDAD DE COMUNICACION SOCIAL,NO,CARRERA TECNICA O COMERCIAL,8bb9602f4a18b6886219e7d9b8f3aca7
4,200205231136471198141,JOSE ANTONIO DAVILA ARMENTA,MODIFICACION,23/05/2002,VERIFICADOR,SECRETARIA DE HACIENDA Y CREDITO PUBLICO,OPERACION ADUANERA,NO,LICENCIATURA,3fa51abc51dca570e73bad0c5e1c76c4


In [107]:
df_general['hash'] = df_general.iloc[:,1:].drop(["FECHA_ENVIO", "TIPO_DECLARACION"], axis=1).apply(
    lambda x: hashlib.md5("".join(x).encode()).hexdigest(), axis=1)
df_general_2003['hash'] = df_general_2003.iloc[:,1:].drop(["FECHA_ENVIO", "TIPO_DECLARACION"], axis=1).apply(
    lambda x: hashlib.md5("".join(x).encode()).hexdigest(), axis=1)

In [108]:
hashs_2002 = set(df_general['hash'].unique())
hashs_2003 = set(df_general_2003["hash"].unique())
print("pocentaje de hashs repethashos: ", len(hashs_2002 & hashs_2003) / len(hashs_2002))

pocentaje de hashs repethashos:  0.3698745053921949
